# Moving Average Trading Strategy Analysis

## Overview
This notebook demonstrates a **Moving Average Crossover Strategy**, one of the most popular and effective technical analysis techniques for identifying trend changes and generating trading signals.

### Strategy Logic
The strategy uses two moving averages to identify momentum shifts:
- When the **fast-moving average (MA10) crosses above the slow-moving average (MA20)**, it signals an upward momentum shift → **BUY**
- When the **fast-moving average (MA10) crosses below the slow-moving average (MA20)**, it signals a downward momentum shift → **SELL**

In [16]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import talib as ta

In [17]:
df = pd.read_csv('../../../data/stocks/AAPL.csv')
df

,Date,Open,High,Low,Close,Adj Close,Volume
0,1980-12-12,0.513393,0.515625,0.513393,0.513393,0.406782,117258400
1,1980-12-15,0.488839,0.488839,0.486607,0.486607,0.385558,43971200
2,1980-12-16,0.453125,0.453125,0.450893,0.450893,0.357260,26432000
3,1980-12-17,0.462054,0.464286,0.462054,0.462054,0.366103,21610400
4,1980-12-18,0.475446,0.477679,0.475446,0.475446,0.376715,18362400
...,...,...,...,...,...,...,...
9904,2020-03-26,246.520004,258.679993,246.360001,258.440002,258.440002,63021800
9905,2020-03-27,252.750000,255.869995,247.050003,247.740005,247.740005,51054200
9906,2020-03-30,250.740005,255.520004,249.399994,254.809998,254.809998,41994100
9907,2020-03-31,255.600006,262.489990,252.000000,254.289993,254.289993,49250500


## Moving Average (MA) Strategy: Technical Analysis Fundamentals

### What are Moving Averages?
Moving Averages are one of the most fundamental and widely-used technical indicators in trading. They smooth out price data by calculating the average price over a specified number of periods, helping traders identify trends and potential buy/sell signals.

**Key Characteristics:**
- **MA10** (10-period): Fast-moving average, responds quickly to price changes, useful for short-term trends
- **MA20** (20-period): Medium-term average, balances responsiveness and stability
- **MA50** (50-period): Slow-moving average, identifies long-term trend direction

### Moving Average Crossover Strategy
This strategy uses the intersection of two moving averages to generate trading signals:
- **Buy Signal (Golden Cross)**: When the faster MA (MA10) crosses above the slower MA (MA20), indicating upward momentum
- **Sell Signal (Death Cross)**: When the faster MA (MA10) crosses below the slower MA (MA20), indicating downward momentum

This is a classic momentum-based strategy that helps traders capitalize on trend changes.

In [18]:
# Calculate Moving Averages using TA-Lib
# MA10: 10-day Simple Moving Average - responds quickly to recent price movements
# MA20: 20-day Simple Moving Average - moderate responsiveness for medium-term trends
# MA50: 50-day Simple Moving Average - identifies long-term trend direction
df['MA10'] = ta.MA(df['Close'], timeperiod=10)
df['MA20'] = ta.MA(df['Close'], timeperiod=20)
df['MA50'] = ta.MA(df['Close'], timeperiod=50)

# Generate Trading Signals using MA10/MA20 Crossover Strategy
# Signal = 1 when MA10 > MA20 (bullish trend), 0 otherwise (bearish/neutral)
df['Signal'] = np.where(df['MA10'] > df['MA20'], 1, 0)

# Detect Exact Entry Points (crossover events)
# Entry = +1: Golden Cross (MA10 crosses above MA20) - BUY SIGNAL
# Entry = -1: Death Cross (MA10 crosses below MA20) - SELL SIGNAL
# Entry = 0: No crossover event on this day (continuation)
df['Entry'] = df['Signal'].diff()

In [19]:
df

,Date,Open,High,Low,Close,Adj Close,Volume,MA10,MA20,MA50,Signal,Entry
0,1980-12-12,0.513393,0.515625,0.513393,0.513393,0.406782,117258400,NaN,NaN,NaN,0,NaN
1,1980-12-15,0.488839,0.488839,0.486607,0.486607,0.385558,43971200,NaN,NaN,NaN,0,0.0
2,1980-12-16,0.453125,0.453125,0.450893,0.450893,0.357260,26432000,NaN,NaN,NaN,0,0.0
3,1980-12-17,0.462054,0.464286,0.462054,0.462054,0.366103,21610400,NaN,NaN,NaN,0,0.0
4,1980-12-18,0.475446,0.477679,0.475446,0.475446,0.376715,18362400,NaN,NaN,NaN,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
9904,2020-03-26,246.520004,258.679993,246.360001,258.440002,258.440002,63021800,246.894002,264.514500,294.6510,0,0.0
9905,2020-03-27,252.750000,255.869995,247.050003,247.740005,247.740005,51054200,243.871002,263.233501,293.3790,0,0.0
9906,2020-03-30,250.740005,255.520004,249.399994,254.809998,254.809998,41994100,245.131001,261.033501,292.1704,0,0.0
9907,2020-03-31,255.600006,262.489990,252.000000,254.289993,254.289993,49250500,245.274001,259.282001,290.8816,0,0.0


## Interactive Chart Visualization

### Visualization Components

We'll create an interactive candlestick chart that combines:
1. **Candlestick Bars**: Show daily OHLC (Open, High, Low, Close) price movements
   - Green candles: Close > Open (bullish)
   - Red candles: Close < Open (bearish)

2. **Three Moving Averages** (Trend Lines):
   - **Blue line (MA10)**: Fast momentum indicator
   - **Green line (MA20)**: Medium-term trend
   - **Red line (MA50)**: Long-term trend direction

3. **Trading Signals** (Markers):
   - **Green Triangles (▲)**: Buy signals (where to enter long positions)
   - **Red Triangles (▼)**: Sell signals (where to exit or enter short positions)

This combined view helps identify trend changes, momentum shifts, and optimal entry/exit points.

In [20]:
# Create the base candlestick chart
# This shows OHLC data where each candle represents one trading day
fig = go.Figure(
    data=go.Candlestick(
        x=df['Date'],
        open=df['Open'],      # Opening price of the day
        high=df['High'],      # Highest price reached during the day
        low=df['Low'],        # Lowest price reached during the day
        close=df['Close'],    # Closing price at end of day
        name='AAPL'
    )
)

# ===== Add Moving Averages to the Chart =====

# MA10 (Fast Moving Average) - Blue line
# Responds quickly to price changes, useful for identifying short-term momentum
fig.add_trace(
    go.Scatter(
        name='MA10',
        x=df['Date'],
        y=df['MA10'],
        line=dict(color='blue', width=1)
    )
)

# MA20 (Medium Moving Average) - Green line
# Balanced between responsiveness and stability, shows medium-term trends
fig.add_trace(
    go.Scatter(
        name='MA20',
        x=df['Date'],
        y=df['MA20'],
        line=dict(color='green', width=1)
    )
)

# MA50 (Slow Moving Average) - Red line
# Identifies the overall long-term trend direction, filters out noise
fig.add_trace(
    go.Scatter(
        name='MA50',
        x=df['Date'],
        y=df['MA50'],
        line=dict(color='red', width=1)
    )
)

# ===== Add Buy Signals (Green Triangles) =====
# Buy signals occur when Entry == 1 (MA10 crosses above MA20 - Golden Cross)
# These are positioned slightly below the price (0.98 multiplier) for visibility
fig.add_trace(go.Scatter(
    x=df[df['Entry'] == 1]['Date'],
    y=df[df['Entry'] == 1]['Low'] * 0.98,  # Position 2% below the low price
    mode='markers',
    marker=dict(
        symbol='triangle-up',  # Upward-pointing triangle
        size=12,
        color='green'
    ),
    name='Buy Signal'
))

# ===== Add Sell Signals (Red Triangles) =====
# Sell signals occur when Entry == -1 (MA10 crosses below MA20 - Death Cross)
# These are positioned slightly above the price (1.02 multiplier) for visibility
fig.add_trace(go.Scatter(
    x=df[df['Entry'] == -1]['Date'],
    y=df[df['Entry'] == -1]['High'] * 1.02,  # Position 2% above the high price
    mode='markers',
    marker=dict(
        symbol='triangle-down',  # Downward-pointing triangle
        size=12,
        color='red'
    ),
    name='Sell Signal'
))

# ===== Configure Chart Layout =====
fig.update_layout(
    title='AAPL Technical Analysis (Moving Averages)',
    yaxis_title='Stock Price (USD)',
    hovermode='x unified'  # Show all values for a specific date when hovering
)

fig.update(layout_xaxis_rangeslider_visible=False)
fig.show()

## Interpretation Guide

### Reading the Chart
1. **Candlestick Colors**:
   - Green candles: Bullish (close > open) - price went up
   - Red candles: Bearish (close < open) - price went down

2. **Moving Average Positions**:
   - When MA10 (blue) is above MA20 (green): Bullish bias
   - When MA10 (blue) is below MA20 (green): Bearish bias
   - When all three MAs align in order (10, 20, 50): Strong trend confirmation

3. **Trading Signals**:
   - Green triangle (▲): Buy opportunity - MA10 crossed above MA20
   - Red triangle (▼): Sell opportunity - MA10 crossed below MA20

### Strategy Performance Considerations
- **False Signals**: In choppy/sideways markets, crossovers may produce whipsaws (quick reversals)
- **Lag Effect**: Moving averages are lagging indicators - they respond to price after the move has started
- **Optimization**: Adjust timeperiods (10, 20, 50) based on your trading style:
  - Shorter periods → More signals (higher risk, higher reward)
  - Longer periods → Fewer signals (lower risk, potentially missed opportunities)